In [1]:
import cv2 as cv
import time
import win32gui, win32ui, win32con, win32api
import numpy as np
import dxcam
from ultralytics import YOLO
from IPython.display import display, clear_output
from win32api import GetSystemMetrics
import pydirectinput
import scipy
import ctypes
import math
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

In [2]:
def move(x, y, duration=0.25, absolute=True, interpolate=False, **kwargs):

    if (interpolate):
        print("mouse move {}".format(interpolate))
        current_pixel_coordinates = win32api.GetCursorPos()
        if interpolate:
            current_pixel_coordinates = win32api.GetCursorPos()
            start_coordinates = _to_windows_coordinates(*current_pixel_coordinates)

            end_coordinates = _to_windows_coordinates(x, y)
            print("In interpolate")
            coordinates = _interpolate_mouse_movement(
                start_windows_coordinates=start_coordinates,
                end_windows_coordinates=end_coordinates
            )
            print(coordinates)
        else:
            coordinates = [end_coordinates]

        for x, y in coordinates:
            extra = ctypes.c_ulong(0)
            ii_ = Input_I()
            ii_.mi = MouseInput(x, y, 0, (0x0001 | 0x8000), 0, ctypes.pointer(extra))
            x = Input(ctypes.c_ulong(0), ii_)
            ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

            time.sleep(duration / len(coordinates))
    else:

        x = int(x)
        y = int(y)

        coordinates = _interpolate_mouse_movement(
            start_windows_coordinates=(0, 0),
            end_windows_coordinates=(x, y)
        )

        for x, y in coordinates:
            extra = ctypes.c_ulong(0)
            ii_ = Input_I()
            ii_.mi = MouseInput(x, y, 0, 0x0001, 0, ctypes.pointer(extra))
            x = Input(ctypes.c_ulong(0), ii_)
            ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

            time.sleep(duration / len(coordinates))


def _to_windows_coordinates(x=0, y=0):
    display_width = win32api.GetSystemMetrics(0)
    display_height = win32api.GetSystemMetrics(1)

    windows_x = (x * 65535) // display_width
    windows_y = (y * 65535) // display_height

    return windows_x, windows_y

def _interpolate_mouse_movement(start_windows_coordinates, end_windows_coordinates, steps=20):
    x_coordinates = [start_windows_coordinates[0], end_windows_coordinates[0]]
    y_coordinates = [start_windows_coordinates[1], end_windows_coordinates[1]]

    if x_coordinates[0] == x_coordinates[1]:
        x_coordinates[1] += 1

    if y_coordinates[0] == y_coordinates[1]:
        y_coordinates[1] += 1

    interpolation_func = scipy.interpolate.interp1d(x_coordinates, y_coordinates)

    intermediate_x_coordinates = np.linspace(start_windows_coordinates[0], end_windows_coordinates[0], steps + 1)[1:]
    coordinates = list(map(lambda x: (int(round(x)), int(interpolation_func(x))), intermediate_x_coordinates))

In [3]:
#BASIC CONFIG#
VERBOSE=False
FPS=True

TRACK =False
PREDICT=True
SAHI=False

FULL_SCREEN=False


WINDOW=False

CONFIDENCE_THRESHOLD=0.5
IOU=0.5

dw = win32api.GetSystemMetrics(0) #display width 
dh = win32api.GetSystemMetrics(1) #display height 


AIM_ASSIST=True
TRIGGER=True

#CHOOSE ONE OF:
ACTIVE_WHEN_KEY_PRESSED=''
ALWAYS_ACTIVE=False


#AIM ASSIST CONFIG(needed only if AIM_ASSIST=True)#
LEFT=int(dw*0.4427)
TOP=int(dh*0.25)
RIGHT=int(dw*0.573)
BOTTOM=int(dh*0.833)


In [4]:
LEFT,TOP,RIGHT,BOTTOM

(849, 270, 1100, 899)

In [5]:
dc = win32gui.GetDC(0)
dcObj = win32ui.CreateDCFromHandle(dc)
hwnd = win32gui.WindowFromPoint((0,0))
monitor = (0, 0, dw, dh)

b1 = win32ui.CreateBrush()
b1.CreateSolidBrush(win32api.GetSysColor(0))



def box_in_center(box):
    if box[0]+LEFT<dw/2 and box[2]+LEFT>dw/2 and box[1]+TOP<dh/2 and box[3]+TOP>dh/2:
        return True
    
    return False
    

def find_closest(r):
    min_distance=1000000
    closest=[]
    
    for box in r.boxes.xywh:
        dist=math.dist([dw/2,dh/2],[ box[0]+LEFT,box[1]+TOP] )
        if dist<min_distance:
            min_distance=dist
            closest=box[0]+LEFT,box[1]+TOP-box[3]*0.35
            
    return closest


def draw_box(left,top,right,bottom):
    dcObj.FrameRect((LEFT, TOP, RIGHT, BOTTOM), b1)
    dcObj.FrameRect((left+LEFT,top,right+LEFT,bottom), b1)
    win32gui.InvalidateRect(hwnd, monitor, True)


In [6]:
camera = dxcam.create(output_color="BGR")
camera.start(region=(LEFT, TOP, RIGHT, BOTTOM))

yolo_model = YOLO("best.pt") 

sahi_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path="best.pt",
    confidence_threshold=CONFIDENCE_THRESHOLD,
    device="cuda:0", # or 'cuda:0'
    image_size=640,
    
)


frames_counter=0
all_time=0

In [7]:
while(True):
    # if win32api.GetKeyState(0x02)<0:
    clear_output(wait=True)
    start= time.time()
    
    screenshot = camera.get_latest_frame()

    # if SAHI:
    #     result= get_sliced_prediction(
    #         screenshot,
    #         sahi_model,
    #         slice_height = 256,
    #         slice_width = 256,
    #         overlap_height_ratio = 0.2,
    #         overlap_width_ratio = 0.2,
    #         verbose:True
    #     )

    #     for r in result.to_coco_annotations():
    #         if AIM_ASSIST:
                
    #         if TRIGGER:

    #         if VERBOSE:
                
    #     continue
        

    if TRACK:
        result=yolo_model.track(screenshot,imgsz=640, conf=CONFIDENCE_THRESHOLD,verbose=False,iou=IOU)
    elif PREDICT:
        result=yolo_model.predict(screenshot,imgsz=640, conf=CONFIDENCE_THRESHOLD,verbose=False,iou=IOU)
        
    for r in result:
        if AIM_ASSIST:
            closest=find_closest(r)
            if len(closest)>0 and  win32api.GetKeyState(0x02)<0:
                
                # Pixel difference between crosshair(center) and the closest object
                x = closest[0] - dw/2
                y = closest[1] - dh/2 
        
                # Move mouse and shoot
                scale = 0.3
                x = int(x * scale)
                y = int(y * scale)
                win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, x, y, 0, 0)
                # time.sleep(0.05)
                        
        if TRIGGER: 
            for box in r.boxes.xyxy:
                if len(box) and  win32api.GetKeyState(0x02)<0:
                    if box_in_center(box):
                        pydirectinput.click(duration=0.4,_pause=False)
                        
        
        if VERBOSE:
            for box in r.boxes.xyxy:
                box=[int(x) for x in box]
                draw_box(*box)
                
            display('Number of persons: '+str(len(r.boxes.xyxy)) )
            display('bboxes in xyxy format: '+str(r.boxes.xyxy) )
            
    
    # cv.imshow('Computer Vision', screenshot)
    
    if FPS: 
        display('FPS: '+str(1/(time.time()-start+0.0015)) )
        frames_counter+=1
        all_time+=1/(time.time()-start)
        display('Avg FPS: '+str(all_time/frames_counter)) 
        

    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        camera.stop()
        break

KeyboardInterrupt: 

In [ ]:
win32api.GetCursorPos()

In [ ]:
win32api.GetSystemMetrics(1)

In [ ]:
pydirectinput.moveRel(None, 100)

In [ ]:
win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, 100, 200, 0, 0)